In [1]:
from dotenv import find_dotenv,load_dotenv
load_dotenv(find_dotenv())

True

# 混合检索召回
`EnsembleRetriever`将检索器列表作为输入，并集成它们的`get_relevance _documents()`方法的结果，并基于互反秩融合算法对结果重新排序。
通过利用不同算法的优势，`EnsembleRetriever`可以获得比任何单一算法更好的性能。
最常见的模式是将稀疏检索器(如BM25)与密集检索器(如嵌入相似度)结合起来，因为它们的优势是互补的。它也被称为“混合搜索”。稀疏检索器擅长根据关键词找到相关文档，而密集检索器擅长根据语义相似度找到相关文档。

In [2]:
%pip install --upgrade --quiet  rank_bm25 > /dev/null

Note: you may need to restart the kernel to use updated packages.


系统找不到指定的路径。


In [3]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [4]:
doc_list_1 = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)
bm25_retriever.k = 2

doc_list_2 = [
    "You like apples",
    "You like oranges",
]

embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_texts(
    doc_list_2, embedding, metadatas=[{"source": 2}] * len(doc_list_2)
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [5]:
docs = ensemble_retriever.invoke("apples")
docs

[Document(page_content='I like apples', metadata={'source': 1}),
 Document(page_content='You like apples', metadata={'source': 2}),
 Document(page_content='Apples and oranges are fruits', metadata={'source': 1}),
 Document(page_content='You like oranges', metadata={'source': 2})]

## 运行时配置
我们还可以在运行时配置检索器。为了做到这一点，我们需要将字段标记为可配置的

In [6]:
from langchain_core.runnables import ConfigurableField

In [7]:
"""
问题还没解决
"""
faiss_retriever = faiss_vectorstore.as_retriever(
    search_kwargs={"k": 2}
).configurable_fields(  # 添加配置后会导致EnsembleRetriever报错，如何解决
    search_kwargs=ConfigurableField(
        id="search_kwargs_faiss",
        name="Search Kwargs",
        description="The search kwargs to use",
    )
)

In [8]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
) # 错误未解决成功

ValidationError: 1 validation error for EnsembleRetriever
retrievers -> 1
  Can't instantiate abstract class BaseRetriever with abstract method _get_relevant_documents (type=type_error)

In [9]:
config = {"configurable": {"search_kwargs_faiss": {"k": 1}}}
docs = ensemble_retriever.invoke("apples", config=config)
docs

[Document(page_content='I like apples', metadata={'source': 1}),
 Document(page_content='You like apples', metadata={'source': 2}),
 Document(page_content='Apples and oranges are fruits', metadata={'source': 1}),
 Document(page_content='You like oranges', metadata={'source': 2})]